In [1]:
import pandas 
import opensoundscape
from opensoundscape.audio import Audio
import warnings
import numpy as np
import pandas as pd
from scipy.signal import correlate, correlation_lags
import statsmodels.api as sm

/opt/anaconda3/envs/poetry_py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### try with a real example
from pathlib import Path
from opensoundscape.torch.models.cnn import load_model
import pandas as pd
from opensoundscape.localization import Localizer
folder = Path("/Users/LOF19/Documents/kitzes_projects/LOCA_class/LOCA22_rotation/classifier/2022_10_06/")
aru_coords = pd.read_csv(folder / "aru_coords.csv", index_col=0)
SAMPLE_RATE = 44100

files = list(Path("/Users/LOF19/Documents/kitzes_projects/LOCA_class/LOCA22_rotation/synchronized").glob("*/*.WAV"))

preds = pd.read_csv(folder / "preds.csv", index_col=[0,1,2])

# add another column for testing purposes 
preds["test"] =  np.random.rand(len(preds)) - 0.8

# Moved files from snowy so
# change paths to files
old_dir = "/home/lfh/projects/loca/LOCA22_rotation/synchronized/"
new_dir = "/Users/LOF19/Documents/kitzes_projects/LOCA_class/LOCA22_rotation/synchronized/"

aru_coords.index = [str(i).replace(old_dir, new_dir) for i in aru_coords.index] 
preds.index.set_levels([str(i).replace(old_dir, new_dir) for i in preds.index.levels[0]], level=0, inplace=True) 

/var/folders/_s/zm3gz4x52glfbflc25_3rp3w0000gq/T/ipykernel_34441/2174934504.py:23: FutureWarning: inplace is deprecated and will be removed in a future version.
  preds.index.set_levels([str(i).replace(old_dir, new_dir) for i in preds.index.levels[0]], level=0, inplace=True)


In [9]:
loca_class = Localizer(
    files = files,
    predictions=preds,
    aru_coords=aru_coords, 
    sample_rate=44100, 
    min_number_of_receivers=4, 
    max_distance_between_receivers=150,
    bandpass_ranges={"COYE":(1000, 10000), "NOT_COYE" : (1000, 10000)},
    max_delay=0.5,
    cc_threshold=0.001,
    thresholds= {"COYE":0.5, "NOT_COYE": 100})

/Users/LOF19/Dev/opensoundscape/opensoundscape/localization.py:68: UserWarning: WARNING: Not all classes have corresponding bandpass ranges. Default behavior will be to not bandpass before cross-correlation for classes that do not have a corresponding bandpass range.
  warnings.warn(
/Users/LOF19/Dev/opensoundscape/opensoundscape/localization.py:76: UserWarning: WARNING: Not all classes have corresponding thresholds. Default behavior will be to drop classes that do not have a corresponding threshold.
  warnings.warn(


In [10]:
loca_class.predictions

COYE  \
file                                               start_time end_time              
/Users/LOF19/Documents/kitzes_projects/LOCA_cla... 0.0        3.0      -13.316370   
                                                   3.0        6.0       -5.374636   
                                                   6.0        9.0      -10.286495   
                                                   9.0        12.0      -8.546298   
                                                   12.0       15.0      -8.124673   
...                                                                           ...   
/Users/LOF19/Documents/kitzes_projects/LOCA_cla... 3582.0     3585.0    -6.739721   
                                                   3585.0     3588.0    -7.965528   
                                                   3588.0     3591.0    -7.629130   
                                                   3591.0     3594.0    -8.809423   
                                                   3594.0     3597.0    -6.776921   

                                                                         NOT_COYE  \
file                                               start_time end_time              
/Users/LOF19/Documents/kitzes_projects/LOCA_cla... 0.0        3.0       13.161119   
                                                   3.0        6.0        5.358257   
                                                   6.0        9.0       10.199137   
                                                   9.0        12.0       8.701430   
                                                   12.0       15.0       8.188313   
...                                                                           ...   
/Users/LOF19/Documents/kitzes_projects/LOCA_cla... 3582.0     3585.0     6.720416   
                                                   3585.0     3588.0     7.885397   
                                                   3588.0     3591.0     7.497522   
                                                   3591.0     3594.0     8.669293   
                                                   3594.0     3597.0     6.760019   

                                                                            test  
file                                               start_time end_time            
/Users/LOF19/Documents/kitzes_projects/LOCA_cla... 0.0        3.0       0.009408  
                                                   3.0        6.0      -0.252431  
                                                   6.0        9.0      -0.164132  
                                                   9.0        12.0     -0.513359  
                                                   12.0       15.0     -0.133604  
...                                                                          ...  
/Users/LOF19/Documents/kitzes_projects/LOCA_cla... 3582.0     3585.0   -0.708278  
                                                   3585.0     3588.0   -0.158344  
                                                   3588.0     3591.0   -0.211683  
                                                   3591.0     3594.0   -0.577605  
                                                   3594.0     3597.0    0.190444  

[288122 rows x 3 columns]

In [11]:
loca_class.threshold_predictions()

,time,reference_file,other_files,species
0,"(426.0, 429.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE
1,"(426.0, 429.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE
2,"(426.0, 429.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE
3,"(426.0, 429.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE
4,"(2334.0, 2337.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE
5,"(2334.0, 2337.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE
6,"(2334.0, 2337.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE
7,"(2973.0, 2976.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE
8,"(2973.0, 2976.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE
9,"(2973.0, 2976.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE


In [12]:
loca_class.cross_correlate()

,time,reference_file,other_files,species,cross_correlations,time_delays
0,"(426.0, 429.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.008886375673271702, 0.04819709213300328, 0....","[6.802721088435374e-05, -0.07095238095238095, ..."
1,"(426.0, 429.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.008886375673271702, 0.006146023336291264, 0...","[-6.802721088435374e-05, 0.4783219954648526, 0..."
2,"(426.0, 429.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.048197092133003265, 0.0061460233362912635, ...","[0.07095238095238095, -0.4783219954648526, 0.0..."
3,"(426.0, 429.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.020745139158917347, 0.0057958456384879925, ...","[0.04217687074829932, -0.1765079365079365, -0...."
4,"(2334.0, 2337.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.005132590117862276, 0.006080218895183521, 0...","[0.40938775510204084, 0.44424036281179136, -0...."
5,"(2334.0, 2337.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.005132590117862276, 0.010039040381242578, 0...","[-0.40938775510204084, 0.00036281179138321996,..."
6,"(2334.0, 2337.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.005900160750755415, 0.006535932637957149, 0...","[0.33435374149659863, 0.2724716553287982, -0.0..."
7,"(2973.0, 2976.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.02996136665816906, 0.032307296806943164, 0....","[0.12988662131519274, -0.01748299319727891, 0...."
8,"(2973.0, 2976.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.02996136665816906, 0.037081185826614474, 0....","[-0.12988662131519274, -0.1473922902494331, -0..."
9,"(2973.0, 2976.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.032307296806943164, 0.03708118582661447, 0....","[0.01748299319727891, 0.1473922902494331, 0.07..."


In [13]:
loca_class.filter_cross_correlations()

0 rows deleted


,time,reference_file,other_files,species,cross_correlations,time_delays
0,"(426.0, 429.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.008886375673271702, 0.04819709213300328, 0....","[6.802721088435374e-05, -0.07095238095238095, ..."
1,"(426.0, 429.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.008886375673271702, 0.006146023336291264, 0...","[-6.802721088435374e-05, 0.4783219954648526, 0..."
2,"(426.0, 429.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.048197092133003265, 0.0061460233362912635, ...","[0.07095238095238095, -0.4783219954648526, 0.0..."
3,"(426.0, 429.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.020745139158917347, 0.0057958456384879925, ...","[0.04217687074829932, -0.1765079365079365, -0...."
4,"(2334.0, 2337.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.005132590117862276, 0.006080218895183521, 0...","[0.40938775510204084, 0.44424036281179136, -0...."
5,"(2334.0, 2337.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.005132590117862276, 0.010039040381242578, 0...","[-0.40938775510204084, 0.00036281179138321996,..."
6,"(2334.0, 2337.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.005900160750755415, 0.006535932637957149, 0...","[0.33435374149659863, 0.2724716553287982, -0.0..."
7,"(2973.0, 2976.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.02996136665816906, 0.032307296806943164, 0....","[0.12988662131519274, -0.01748299319727891, 0...."
8,"(2973.0, 2976.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.02996136665816906, 0.037081185826614474, 0....","[-0.12988662131519274, -0.1473922902494331, -0..."
9,"(2973.0, 2976.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.032307296806943164, 0.03708118582661447, 0....","[0.01748299319727891, 0.1473922902494331, 0.07..."


In [14]:
loca_class.localize()

/opt/anaconda3/envs/poetry_py39/lib/python3.9/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
/opt/anaconda3/envs/poetry_py39/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1765: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/opt/anaconda3/envs/poetry_py39/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1765: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/opt/anaconda3/envs/poetry_py39/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1687: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/opt/anaconda3/envs/poetry_py39/lib/python3.9/site-packages/st

,time,reference_file,other_files,species,cross_correlations,time_delays,predicted_x,predicted_y,gillette_error
0,"(426.0, 429.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.008886375673271702, 0.04819709213300328, 0....","[6.802721088435374e-05, -0.07095238095238095, ...",665115.660211,4.461538e+06,Error
1,"(426.0, 429.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.008886375673271702, 0.006146023336291264, 0...","[-6.802721088435374e-05, 0.4783219954648526, 0...",665153.569523,4.461487e+06,Error
2,"(426.0, 429.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.048197092133003265, 0.0061460233362912635, ...","[0.07095238095238095, -0.4783219954648526, 0.0...",665085.676994,4.461447e+06,Error
3,"(426.0, 429.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.020745139158917347, 0.0057958456384879925, ...","[0.04217687074829932, -0.1765079365079365, -0....",665117.978692,4.461545e+06,Error
4,"(2334.0, 2337.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.005132590117862276, 0.006080218895183521, 0...","[0.40938775510204084, 0.44424036281179136, -0....",665264.382532,4.461479e+06,Error
5,"(2334.0, 2337.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.005132590117862276, 0.010039040381242578, 0...","[-0.40938775510204084, 0.00036281179138321996,...",665275.664765,4.461572e+06,Error
6,"(2334.0, 2337.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.005900160750755415, 0.006535932637957149, 0...","[0.33435374149659863, 0.2724716553287982, -0.0...",665305.735848,4.461595e+06,Error
7,"(2973.0, 2976.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.02996136665816906, 0.032307296806943164, 0....","[0.12988662131519274, -0.01748299319727891, 0....",665096.153321,4.461510e+06,Error
8,"(2973.0, 2976.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.02996136665816906, 0.037081185826614474, 0....","[-0.12988662131519274, -0.1473922902494331, -0...",665096.157516,4.461510e+06,Error
9,"(2973.0, 2976.0)",/Users/LOF19/Documents/kitzes_projects/LOCA_cl...,[/Users/LOF19/Documents/kitzes_projects/LOCA_c...,COYE,"[0.032307296806943164, 0.03708118582661447, 0....","[0.01748299319727891, 0.1473922902494331, 0.07...",665096.162470,4.461510e+06,Error


In [ ]:
[len(x) for x in loca_class.locations["cross_correlations"]]

<Audio(samples=(1920000,), sample_rate=32000)>